In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from elasticsearch import Elasticsearch, helpers
import configparser
from datetime import datetime
import json

In [2]:
config = configparser.ConfigParser()
config.read('example.ini')

['example.ini']

In [3]:
es = Elasticsearch(
    cloud_id=config['ELASTIC']['cloud_id'],
    http_auth=(config['ELASTIC']['user'], config['ELASTIC']['password'])
)

C:\Users\202210829\AppData\Local\Temp\ipykernel_11872\2962435662.py:1: DeprecationWarning: The 'http_auth' parameter is deprecated. Use 'basic_auth' or 'bearer_auth' parameters instead
  es = Elasticsearch(


In [4]:
category_num = {
                "로맨스":"017001046",
                "BL": "017001064",
                "만화":"017001038",
                "판타지 무협": "017001049",
                "소설": "017001045",
                "경제 경영" : "017001047",                  
                "라이트노벨" : "017001063",
                "에세이 시" : "017001050",
                "인문" : "017001048",
                "사회 정치" : "017001051",
                "자기계발" : "017001054",
                "역사" : "017001052",
                "종교" : "017001053",
                "예술 대중문화" : "017001056",
                "자연과학" : "017001055",
                "가정 살림" : "017001042",
                "건강 취미 여행" : "017001043",
                "어린이 유아" : "017001058",
                "청소년" : "017001059",
                "국어 외국어" : "017001060",
                "IT 모발일" : "017001040",
                "대학교재" : "017001041",
                "수험서 자격증" : "017001061",
                "잡지" : "017001036",
                "해외원서": "017001032",
                "오디오북": "017001032",
                "크레마 (전자책 단말기)": "017001032"
                
}

In [5]:
name_mapping = {
                "로맨스":"문학",
                "BL": "문학",
                "만화":"문학",
                "판타지 무협": "문학",
                "소설": "문학",
                "경제 경영" : "경제/비즈니스",
                "라이트노벨" : "문학",
                "에세이 시" : "에세이/산문",
                "인문" : "인문",
                "사회 정치" : "인문",
                "자기계발" : "인문",
                "역사" : "인문",
                "종교" : "인문",
                "예술 대중문화" : "가정/생활",
                "자연과학" : "자연/과학",
                "가정 살림" : "가정/생활",
                "건강 취미 여행" : "가정/생활",
                "어린이 유아" : "가정/생활",
                "청소년" : "가정/생활",
                "국어 외국어" : "외국어",
                "IT 모발일" : "컴퓨터/인터넷",
                "대학교재" : "강의지원도서",
                "수험서 자격증" : "강의지원도서",
                "잡지" : "국외 eBook",
                "해외원서": "국외 eBook",
                "오디오북": "국외 eBook",
                "크레마 (전자책 단말기)": "국외 eBook"
                
}

In [6]:
def create_url(category_number):
    url = "http://www.yes24.com/24/category/bestseller?CategoryNumber={0}&sumgb=07&FetchSize=80".format(category_number)

    return url
    

In [7]:
def create_url2(category_number):
    url = "http://www.yes24.com/24/category/bestseller?CategoryNumber={0}&sumgb=07&FetchSize=20&PageNumber=2".format(category_number)

    return url
    

In [8]:
bookss = []
for key, value in category_num.items():
    rank = 0
    crolling_url = create_url(value)
    html = requests.get(crolling_url).text
    soup = BeautifulSoup(html, 'html.parser')
    books = soup.select('td.goodsTxtInfo')
    
    
    

    for book in books:
        title = book.select('p:nth-child(1)')[0].text
        price = book.select('p:nth-child(3) > span.priceB')[0].text
        info = book.select('div.aupu')[0].text
        if info.count('|') ==3:
            pass
        else:
            writer = info.split('|')[0]
        
            publisher = info.split('|')[1]

            date = info.split('|')[2]

        stars = book.select('p.review > img')
        
        star_content = 0
        star_edit = 0

        if stars==None:
                    
                star_content = 0
                star_edit = 0
                
        else:

        
            for star in stars[:5]:
                
                if star['src'] == 'http://image.yes24.com/sysimage/interface/StarOn.gif':
                    
                    star_content += 1

            for star in stars[5:]:

                if star['src'] == 'http://image.yes24.com/sysimage/interface/StarOn.gif':
        
                    star_edit += 1
        

        
        rank +=1
        bookss.append([rank,title,writer, publisher, date,price, star_content,star_edit,key]) # 80 add

    crolling_url2 = create_url2(value)
    html2 = requests.get(crolling_url2).text
    soup2 = BeautifulSoup(html2, 'html.parser')
    books2 = soup2.select('td.goodsTxtInfo')

        
    for book in books2:
        title = book.select('p:nth-child(1)')[0].text
        price = book.select('p:nth-child(3) > span.priceB')[0].text
        info = book.select('div.aupu')[0].text
        
        if info.count('|') ==3:
            pass
        else:
            writer = info.split('|')[0]
        
            publisher = info.split('|')[1]

            date = info.split('|')[2]

        publisher = info.split('|')[1]

        date = info.split('|')[2]

        stars = book.select('p.review > img')
        
        star_content = 0
        star_edit = 0

        if stars==None:
                    
                star_content = 0
                star_edit = 0
                
        else:

        
            for star in stars[:5]:
                
                if star['src'] == 'http://image.yes24.com/sysimage/interface/StarOn.gif':
                    
                    star_content += 1

            for star in stars[5:]:

                if star['src'] == 'http://image.yes24.com/sysimage/interface/StarOn.gif':
        
                    star_edit += 1
        

   
        rank +=1
        
        bookss.append([rank,title,writer, publisher, date,price, star_content,star_edit,key]) # 20 add


        



    


In [9]:
df = pd.DataFrame(bookss, columns=['yes24_rank','제목','yes24_writer', 'yes24_company','yes24_date','yes24_price' ,'yes24_star_content','yes24_star_edit','yes24_gener'])

df['yes24_date'] = df['yes24_date'].str.replace(pat=r'[^A-Za-z0-9가-힣]', repl=r' ', regex=True)
df['제목'] = df['제목'].str.replace(pat=r'[^A-Za-z0-9가-힣]', repl=r' ', regex=True)
df['yes24_writer'] = df['yes24_writer'].str.replace(pat=r'[^A-Za-z0-9가-힣]', repl=r' ', regex=True)
df['yes24_company'] = df['yes24_company'].str.replace(pat=r'[^A-Za-z0-9가-힣]', repl=r' ', regex=True)


for i in ['eBook','EPUB','완결','COMIC','PDF']:
    df['제목'] = df['제목'].str.replace(pat=i, repl=r' ', regex=True)


func = lambda x: name_mapping.get(x, x)


df['sm_gener'] = df.yes24_gener.map(func)
df['time_stamp'] = datetime.now()

In [10]:
df

,yes24_rank,제목,yes24_writer,yes24_company,yes24_date,yes24_price,yes24_star_content,yes24_star_edit,yes24_gener,sm_gener,time_stamp
0,1,조건 만남 삽화본 ...,문정민 저 ...,로아,2021년 05월,"3,000원",4,4,로맨스,문학,2023-02-22 23:13:21.976838
1,2,토굴 공주,함초롱 저 ...,제로노블,2022년 12월,"3,800원",4,4,로맨스,문학,2023-02-22 23:13:21.976838
2,3,세트 오빠 손은 약손 외전 포함 총2권,금나래 저 ...,텐북,2021년 05월,"4,200원",4,4,로맨스,문학,2023-02-22 23:13:21.976838
3,4,세트 마법사가 잠든 사이 총2권,샤아드 저 ...,일랑,2023년 02월,"2,160원",3,3,로맨스,문학,2023-02-22 23:13:21.976838
4,5,오 담에 핀 꽃 개정판 ...,박영 저 ...,플로린,2023년 02월,"3,150원",4,5,로맨스,문학,2023-02-22 23:13:21.976838
...,...,...,...,...,...,...,...,...,...,...,...
2056,14,No Longer at Ease ...,Chinua Achebe 저 ...,Penguin Publishing Group ...,2011년 09월,"11,500원",0,0,크레마 (전자책 단말기),국외 eBook,2023-02-22 23:13:21.976838
2057,15,What to Listen For in Music ...,Aaron Copland 저 ...,Penguin Publishing Group ...,2011년 02월,"11,500원",0,0,크레마 (전자책 단말기),국외 eBook,2023-02-22 23:13:21.976838
2058,16,The Purpose Driven Life ...,Rick Warren 저 ...,Zondervan ...,2012년 10월,"22,500원",0,0,크레마 (전자책 단말기),국외 eBook,2023-02-22 23:13:21.976838
2059,17,The Watsons Go to Birmingham 1963 ...,Christopher Paul Curtis 저 ...,Random House Children s Books ...,2001년 11월,"14,000원",0,0,크레마 (전자책 단말기),국외 eBook,2023-02-22 23:13:21.976838


In [11]:


df['yes24_date'] = df['yes24_date'].str.strip()

df['yes24_date'] = df['yes24_date'].str.replace(pat='년 ', repl = '-',regex=True)
df['yes24_date'] = df['yes24_date'].str.replace(pat='월', repl = '',regex=True)
df['yes24_date'] = df['yes24_date'].str.strip()
df['yes24_date']=pd.to_datetime(df['yes24_date'], format='%Y-%m')


for j in ['원',',']:
    df['yes24_price'] = df['yes24_price'].str.replace(pat=j, repl = '',regex=True)
    df['yes24_price'] = df['yes24_price'].str.strip()
df['yes24_price'] = pd.to_numeric(df['yes24_price'])




In [12]:
df = df[['yes24_rank','제목','yes24_writer', 'yes24_company','yes24_date','yes24_price' ,'yes24_star_content','yes24_star_edit','yes24_gener','sm_gener','time_stamp']]

df.to_csv('yes24.csv', index= False, encoding= 'utf-8')

In [15]:
json = df.to_json(orient = 'records',force_ascii=False)


In [17]:
for i in range(len(df)) :
     es.index(index='yes24_best_100_test',#dd
         document = {
         "yes24_rank" : str(df.loc[i].yes24_rank),
         "제목" : str(df.loc[i].제목),
         "yes24_writer": str(df.loc[i].yes24_writer),
         "yes24_company": str(df.loc[i].yes24_company),
         "yes24_date": str(df.loc[i].yes24_date),
         "yes24_price": str(df.loc[i].yes24_price),
         "yes24_star_content": str(df.loc[i].yes24_star_content),
         "yes24_star_edit": str(df.loc[i].yes24_star_edit),
         "yes24_gener": str(df.loc[i].yes24_gener),
         "time_stamp": str(df.loc[i].time_stamp)
 })
 